####Libraries
***

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import csv
from scipy.stats import norm
import statsmodels.api as sm
from patsy import dmatrices #a Python library for describing statistical models and building Design Matrices using R-like formulas.
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
from collections import defaultdict

In [2]:
!pip install yfinance --upgrade --no-cache-dir
import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 38.7 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


####Load Dataset
***

In [3]:
#φτιάχνω τη λίστα με τα assets μου -> 50 μετοχές 
#διόρθωση : θα βγάλω τυχαία άλλες 4 μετοχές μαζί με την OGN επειδή δεν έχει το ίδιο μέγεθος με τις άλλες 
#βγάζω OGN ,WRK,HST,CE,ACN
assets= ["ABBV","AEP","AIZ","ALLE","AMAT","AMP","AMZN","AVB","AVY",   
  "AXP","BDX","BF-B","BMY","BR","CARR","CDW","CHTR","CNC",   
  "CNP","COP","CTAS","CZR","DG","DPZ","DRE","DXC","META","FTV",   
  "GOOG","GPC","HIG","JPM","KR","PG","PPL","PRU",   
  "PYPL","RE","ROL","ROST","UNH","URI","V","VRSK","XOM"]

In [4]:
#άρα πρέπει να κατεβάσουμε όλα τα adj close για όλες τις τιμές όλων των μετοχών και να κάνουμε απόλυτο ranking και σε 5 κλασεις (targets)
r=pd.DataFrame(index=pdr.get_data_yahoo(assets[0],'2021-01-01','2022-04-01')['Adj Close'].index)

[*********************100%***********************]  1 of 1 completed


In [5]:
#παίρνω τιμές για ένα χρόνο και 3 μήνες
for i in assets:
  # print('i metoxi me onoma:',i,'exei megethos',pdr.get_data_yahoo(i,'2021-01-01','2022-04-01')['Adj Close'].count())
  temp= pdr.get_data_yahoo(i,'2021-01-01','2022-04-01')['Adj Close']
  temp=temp.dropna()
  r[i]=temp.values

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [6]:
#φτιάχνω ένα αντίγραφο με τις τιμές για να το έχω 
rd=pd.DataFrame(r)
rd

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-04 00:00:00-05:00,96.813477,76.819252,128.242004,111.874290,85.672470,182.556000,159.331497,147.581146,148.152328,115.185814,...,70.458298,231.919998,218.876434,37.469650,114.689148,341.345001,229.649994,215.285767,200.593597,37.906910
2021-01-05 00:00:00-05:00,97.814575,76.715622,128.280823,112.089287,88.364822,181.488480,160.925507,147.154129,148.474228,115.800583,...,71.140762,234.910004,217.072525,38.086964,114.787094,336.754639,235.570007,212.072693,199.693878,39.733749
2021-01-06 00:00:00-05:00,96.969620,77.327995,133.911011,114.639877,89.577881,191.901703,156.919006,151.101746,154.150650,120.084427,...,75.355339,226.830002,229.939713,38.714066,119.802986,350.886993,256.549988,210.204163,197.805374,40.747646
2021-01-07 00:00:00-05:00,98.007446,75.264786,135.260315,117.659546,93.256462,194.221146,158.108002,149.042511,158.959000,119.134567,...,75.023338,235.039993,231.590088,39.360771,119.538467,356.512604,264.679993,211.380646,198.705139,41.067341
2021-01-08 00:00:00-05:00,98.521790,74.595879,134.551682,117.249107,94.242668,195.599213,159.134995,150.352066,152.882690,119.252068,...,74.423897,242.460007,229.296829,39.978073,119.920540,354.910919,260.149994,213.001999,198.635910,41.524048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-25 00:00:00-04:00,156.891495,95.333153,183.029373,111.363403,136.376923,311.729828,164.773499,241.656281,171.694946,188.315094,...,118.004997,113.760002,295.426666,33.621418,89.567093,509.607941,363.670013,217.628159,207.255630,83.524574
2022-03-28 00:00:00-04:00,157.513885,96.091171,182.225754,112.533562,136.973938,308.232391,168.990494,244.344177,172.051941,187.741089,...,117.223892,116.169998,294.201538,34.228012,90.897385,509.776794,363.399994,219.959564,210.842743,81.181572
2022-03-29 00:00:00-04:00,157.718109,97.154373,179.884308,114.090469,140.735077,309.342041,169.315002,252.427414,177.406876,188.968292,...,118.004997,121.180000,298.953888,34.953945,94.858521,507.323273,365.200012,227.282578,214.031296,80.750229


In [7]:
rd1= rd.pct_change().shift(-1)
rd1

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-04 00:00:00-05:00,0.010340,-0.001349,0.000303,0.001922,0.031426,-0.005848,0.010004,-0.002893,0.002173,0.005337,...,0.009686,0.012892,-0.008242,0.016475,0.000854,-0.013448,0.025778,-0.014925,-0.004485,0.048193
2021-01-05 00:00:00-05:00,-0.008638,0.007982,0.043890,0.022755,0.013728,0.057377,-0.024897,0.026826,0.038232,0.036993,...,0.059243,-0.034396,0.059276,0.016465,0.043697,0.041966,0.089060,-0.008811,-0.009457,0.025517
2021-01-06 00:00:00-05:00,0.010703,-0.026681,0.010076,0.026340,0.041066,0.012087,0.007577,-0.013628,0.031193,-0.007910,...,-0.004406,0.036194,0.007177,0.016705,-0.002208,0.016033,0.031690,0.005597,0.004549,0.007846
2021-01-07 00:00:00-05:00,0.005248,-0.008887,-0.005239,-0.003488,0.010575,0.007095,0.006496,0.008786,-0.038226,0.000986,...,-0.007990,0.031569,-0.009902,0.015683,0.003196,-0.004493,-0.017115,0.007670,-0.000348,0.011121
2021-01-08 00:00:00-05:00,0.016314,-0.005052,-0.005772,0.002917,0.025115,0.000149,-0.021519,-0.006690,0.029282,-0.005912,...,0.000991,-0.020457,-0.007197,0.002696,-0.012581,-0.007265,-0.010379,-0.011882,-0.015082,0.030356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-25 00:00:00-04:00,0.003967,0.007951,-0.004391,0.010508,0.004378,-0.011219,0.025593,0.011123,0.002079,-0.003048,...,-0.006619,0.021185,-0.004147,0.018042,0.014852,0.000331,-0.000742,0.010713,0.017308,-0.028052
2022-03-28 00:00:00-04:00,0.001297,0.011065,-0.012849,0.013835,0.027459,0.003600,0.001920,0.033081,0.031124,0.006537,...,0.006663,0.043126,0.016153,0.021209,0.043578,-0.004813,0.004953,0.033293,0.015123,-0.005313
2022-03-29 00:00:00-04:00,0.009681,0.007397,0.004743,-0.014776,-0.039875,-0.005189,-0.017801,-0.018389,-0.011571,-0.006180,...,-0.004633,-0.022198,0.007072,0.008250,-0.030769,0.019756,-0.016457,-0.018280,0.010102,0.017118


In [8]:
#first: ranks assigned in order they appear in the array
rd2=rd1.rank(axis=1,method='first')
rd2=rd2.dropna()
rd2.astype(int)
jk=rd2.copy()
jk.astype(int)

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-04 00:00:00-05:00,34,14,15,21,41,9,33,12,22,25,...,32,36,6,38,17,5,40,3,11,44
2021-01-05 00:00:00-05:00,8,14,37,25,20,40,3,28,31,29,...,41,1,42,21,36,35,44,7,6,26
2021-01-06 00:00:00-05:00,25,2,23,36,44,28,20,3,40,8,...,9,43,19,30,11,29,41,18,16,21
2021-01-07 00:00:00-05:00,28,8,13,17,37,31,30,34,1,22,...,11,45,6,42,26,14,3,33,19,38
2021-01-08 00:00:00-05:00,39,23,22,29,42,26,5,20,43,21,...,27,7,16,28,10,15,13,11,9,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-24 00:00:00-04:00,21,16,27,4,3,28,22,40,45,24,...,38,2,37,25,5,8,36,19,7,42
2022-03-25 00:00:00-04:00,22,27,9,31,23,4,45,36,19,13,...,8,42,11,41,38,18,16,32,40,2
2022-03-28 00:00:00-04:00,7,20,2,24,36,11,9,41,39,14,...,15,43,26,33,44,5,12,42,25,4


In [9]:
def rank_data(df1):
  for i in range(len(df1.index)):
    d=df1.loc[df1.index[i]]  #της κάθε μέρας d[1] δίνει μόνο τις τιμές 
    #print(d)
    d=d.sort_values()
    d[0:9]=1
    d[9:18]=2
    d[18:27]=3
    d[27:36]=4
    d[36:45]=5
    df1.loc[df1.index[i]]=d
    #print(df1.loc[df1.index[i]])
    #print(d)
  df1=df1.astype(int)
  return df1
r5=rd2.copy()
rd3=rank_data(r5)
rd3

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-04 00:00:00-05:00,4,2,2,3,5,1,4,2,3,3,...,4,4,1,5,2,1,5,1,2,5
2021-01-05 00:00:00-05:00,1,2,5,3,3,5,1,4,4,4,...,5,1,5,3,4,4,5,1,1,3
2021-01-06 00:00:00-05:00,3,1,3,4,5,4,3,1,5,1,...,1,5,3,4,2,4,5,2,2,3
2021-01-07 00:00:00-05:00,4,1,2,2,5,4,4,4,1,3,...,2,5,1,5,3,2,1,4,3,5
2021-01-08 00:00:00-05:00,5,3,3,4,5,3,1,3,5,3,...,3,1,2,4,2,2,2,2,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-24 00:00:00-04:00,3,2,3,1,1,4,3,5,5,3,...,5,1,5,3,1,1,4,3,1,5
2022-03-25 00:00:00-04:00,3,3,1,4,3,1,5,4,3,2,...,1,5,2,5,5,2,2,4,5,1
2022-03-28 00:00:00-04:00,1,3,1,3,4,2,1,5,5,2,...,2,5,3,4,5,1,2,5,3,1


####Συνάρτηση για τον υπολογισμό του RPS


In [10]:
def RPS_calculation(hist_data, submission, asset_no=44):
    # print('check')
    if hist_data.shape[0]<=asset_no:
        return np.nan    
    # print('check')
    asset_id = pd.unique(hist_data.symbol)
    
    for i in range(len(pd.unique(hist_data.date))):
        if len(hist_data[hist_data.date == pd.unique(hist_data.date)[i]])<len(asset_id):
            for asset in [x for x in asset_id if x not in hist_data[hist_data.date == pd.unique(hist_data.date)[i]].symbol.values]:
                right_price = hist_data[hist_data.symbol==asset].sort_values(by='date')
                right_price = right_price[right_price.date <= pd.unique(hist_data.date)[i]]
                right_price = right_price.price.iloc[-1]
                hist_data = hist_data.append({'date' : pd.unique(hist_data.date)[i],
                                               'symbol' : asset,
                                               'price' : right_price}, ignore_index=True)

    #Compute percentage returns
    asset_id = sorted(asset_id) 

    #Compute percentage returns
    returns = pd.DataFrame(columns = ["ID", "Return"])
    
    min_date = min(hist_data.date)
    # max_date = max(hist_data.date)
    max_date = np.max(hist_data.date)
    
    for i in range(0,len(asset_id)):
        temp = hist_data.loc[hist_data.symbol==asset_id[i]]
        
        open_price = float(temp.loc[temp.date==min_date].price)
        close_price = float(temp.loc[temp.date==max_date].price)
        
        returns = returns.append({'ID': temp.symbol.iloc[0], 
                                'Return': (close_price - open_price)/open_price}, ignore_index=True)

    #Define the relevant position of each asset
    ranking = pd.DataFrame(columns=["ID", "Position", "Return"])
    ranking.ID = list(asset_id)
    ranking.Return = returns.Return
    ranking.Position = ranking.Return.rank(method = 'min')

    #Handle Ties
    Series_per_position = pd.DataFrame(columns=["Position","Series", "Rank", "Rank1", "Rank2","Rank3", "Rank4", "Rank5"])
    Series_per_position.Position = list(pd.unique(ranking.Position.sort_values(ascending=True)))
    temp = ranking.Position.value_counts()
    temp = pd.DataFrame(zip(temp.index, temp), columns = ["Rank", "Occurencies"])
    temp = temp.sort_values(by = ["Rank"],ascending=True)
    Series_per_position.Series = list(temp.Occurencies)
    Series_per_position

    total_ranks = Series_per_position.Position.values[-1]
    for i in range(0,Series_per_position.shape[0]):
    
        start_p = Series_per_position.Position[i]
        end_p = Series_per_position.Position[i] + Series_per_position.Series[i]
        temp = pd.DataFrame(columns = ["Position","Rank", "Rank1", "Rank2", "Rank3", "Rank4","Rank5"])
        temp.Position = list(range(int(start_p),int(end_p)))

        if(temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank=1)
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank1=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank=2)
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank2=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank=3)
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank3=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank=4)
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank4=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank=5)
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank5=1.0)
        temp = temp.fillna(0)
        Series_per_position.iloc[i,2:Series_per_position.shape[1]] = temp.mean(axis = 0).iloc[1:temp.shape[1]]

    Series_per_position = Series_per_position.drop('Series', axis = 1)
    ranking = pd.merge(ranking,Series_per_position, on = "Position")
    ranking = ranking[["ID", "Return", "Position", "Rank", "Rank1", "Rank2", "Rank3", "Rank4", "Rank5"]]
    ranking = ranking.sort_values(["Position"])
    print(ranking)
    #Evaluate submission
    rps_sub = []
    #for aid in list((pd.unique(ranking.ID))):
    for aid in asset_id:

        target = np.cumsum(ranking.loc[ranking.ID==aid].iloc[:,4:9].values).tolist()
        frc = np.cumsum(submission.loc[submission.ID==aid].iloc[:,1:6].values).tolist()
        rps_sub.append(np.mean([(a - b)**2 for a, b in zip(target, frc)]))
    submission["RPS"] = rps_sub
    
    output = {'RPS' : np.mean(rps_sub),
              'details' : submission}
    
    return(output)

####'Εχω τα data έτοιμα και τώρα θα υπολογίσω με πιθανότητες την καινούρια μέθοδο σύμφωνα κάθε φορά με το insample μου.

In [36]:
#Αυτή η συνάρτηση παίρνει τα data και κάνει τη dummy μέτρηση για το διάστημα ενός χρόνου (όσο οριστεί στην ουσία το insample) για μια μετοχή
n=dict() 
def dummy(st,i):
  insample=st.iloc[:-i]
  print('Η ημέρα που πραγματοποιείται η πρόβλεψη είναι:',str(st.iloc[-i:-i+1])[4:15])
  print('Tο insample φτάνει μέχρι και την ημέρα:',str(insample.tail(1))[4:15])
  
  count=defaultdict(int)
  for word in insample.values:
    count[word]+=1
  print(word,count)
  count=sorted(count.items()) #list Η πρώτη τιμή είναι η κλάση και η δεύτερη πόσες φορές είδα τον αριθμό.
  temp=[]
  for k in range(5):
    flag=True
    for l in count:
      if(l[0]==k+1):
        temp.append(l[1]/len(insample))
        flag=False
        break
    if(flag):
      temp.append(0)
  # print(temp,count)
  data={
      'prediction':temp
  }
  forecast=pd.DataFrame(data)
  return forecast
# #για όλες τις μετοχές υπολογίζεται το counter τους
dummy(rd3['ABBV'],60)
# for k in rd3.columns:
#   n[k]=dummy(rd3[k],61)
# n['ABBV']

Η ημέρα που πραγματοποιείται η πρόβλεψη είναι: 
2022-01-04
Tο insample φτάνει μέχρι και την ημέρα: 
2022-01-03
2 defaultdict(<class 'int'>, {4: 61, 1: 42, 3: 58, 5: 47, 2: 45})


,prediction
0,0.166008
1,0.177866
2,0.229249
3,0.241107
4,0.185771


In [ ]:
def pred(st):
  pred=dict()
  for j in st:
    temp=[]
    date=[]
    for i in range(60,1,-1):
      # print(i)
      date.append(rd3[j].iloc[-i:-i+1].index.date[0])
      print('imera provlepsis',rd3[j].iloc[-i:-i+1].index.date[0])
      temp.append(dummy(rd3[j],i)['prediction'].values)
      # pred[rd3[j].name]=temp
    #for the last day 
    date.append(rd3[j].iloc[-1:].index.date[0])
    temp.append(dummy(rd3[j],1)['prediction'].values)
    pred[rd3[j].name]=temp
  # print(date)
  pred['date']=date
  pred_final=pd.DataFrame.from_dict(pred)
  # pred_final['Date']=date
  # pred_final.set_index('Date')
  # print(pred_final)
  return pred_final
t=pred(rd3)

Η έξοδος ροής περικόπηκε στις τελευταίες 5000 γραμμές.
Name: GOOG, dtype: int64
3 defaultdict(<class 'int'>, {4: 61, 2: 48, 5: 54, 1: 41, 3: 65})
imera provlepsis 2022-01-28
TO insample ftanei mexri: Date
2022-01-27    5
Name: GOOG, dtype: int64
5 defaultdict(<class 'int'>, {4: 61, 2: 48, 5: 55, 1: 41, 3: 65})
imera provlepsis 2022-01-31
TO insample ftanei mexri: Date
2022-01-28    4
Name: GOOG, dtype: int64
4 defaultdict(<class 'int'>, {4: 62, 2: 48, 5: 55, 1: 41, 3: 65})
imera provlepsis 2022-02-01
TO insample ftanei mexri: Date
2022-01-31    4
Name: GOOG, dtype: int64
4 defaultdict(<class 'int'>, {4: 63, 2: 48, 5: 55, 1: 41, 3: 65})
imera provlepsis 2022-02-02
TO insample ftanei mexri: Date
2022-02-01    5
Name: GOOG, dtype: int64
5 defaultdict(<class 'int'>, {4: 63, 2: 48, 5: 56, 1: 41, 3: 65})
imera provlepsis 2022-02-03
TO insample ftanei mexri: Date
2022-02-02    1
Name: GOOG, dtype: int64
1 defaultdict(<class 'int'>, {4: 63, 2: 48, 5: 56, 1: 42, 3: 65})
imera provlepsis 2022-02

In [ ]:
t

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM,date
0,"[0.16600790513833993, 0.17786561264822134, 0.2...","[0.1976284584980237, 0.21739130434782608, 0.20...","[0.1422924901185771, 0.2490118577075099, 0.229...","[0.16205533596837945, 0.24110671936758893, 0.1...","[0.31225296442687744, 0.11857707509881422, 0.0...","[0.13438735177865613, 0.22134387351778656, 0.2...","[0.25296442687747034, 0.18181818181818182, 0.1...","[0.15019762845849802, 0.18181818181818182, 0.2...","[0.15810276679841898, 0.20948616600790515, 0.2...","[0.17391304347826086, 0.20948616600790515, 0.2...",...,"[0.3241106719367589, 0.1383399209486166, 0.086...","[0.18972332015810275, 0.20948616600790515, 0.2...","[0.233201581027668, 0.18972332015810275, 0.177...","[0.23715415019762845, 0.18972332015810275, 0.1...","[0.16205533596837945, 0.1976284584980237, 0.24...","[0.2924901185770751, 0.15810276679841898, 0.12...","[0.1857707509881423, 0.2015810276679842, 0.260...","[0.15810276679841898, 0.1857707509881423, 0.26...","[0.2727272727272727, 0.15810276679841898, 0.15...",2022-01-04
1,"[0.16535433070866143, 0.17716535433070865, 0.2...","[0.1968503937007874, 0.21653543307086615, 0.20...","[0.14173228346456693, 0.24803149606299213, 0.2...","[0.16141732283464566, 0.2440944881889764, 0.18...","[0.31496062992125984, 0.11811023622047244, 0.0...","[0.13385826771653545, 0.2204724409448819, 0.21...","[0.25196850393700787, 0.18110236220472442, 0.1...","[0.14960629921259844, 0.18110236220472442, 0.2...","[0.15748031496062992, 0.20866141732283464, 0.2...","[0.1732283464566929, 0.20866141732283464, 0.20...",...,"[0.3228346456692913, 0.14173228346456693, 0.08...","[0.1889763779527559, 0.20866141732283464, 0.22...","[0.23622047244094488, 0.1889763779527559, 0.17...","[0.24015748031496062, 0.1889763779527559, 0.16...","[0.16141732283464566, 0.1968503937007874, 0.24...","[0.2952755905511811, 0.15748031496062992, 0.12...","[0.18503937007874016, 0.20078740157480315, 0.2...","[0.16141732283464566, 0.18503937007874016, 0.2...","[0.27165354330708663, 0.15748031496062992, 0.1...",2022-01-05
2,"[0.16470588235294117, 0.1803921568627451, 0.22...","[0.19607843137254902, 0.2196078431372549, 0.20...","[0.1411764705882353, 0.25098039215686274, 0.22...","[0.16470588235294117, 0.24313725490196078, 0.1...","[0.3137254901960784, 0.11764705882352941, 0.09...","[0.13333333333333333, 0.2196078431372549, 0.21...","[0.25098039215686274, 0.1843137254901961, 0.18...","[0.14901960784313725, 0.1803921568627451, 0.20...","[0.1568627450980392, 0.20784313725490197, 0.25...","[0.17254901960784313, 0.20784313725490197, 0.2...",...,"[0.3215686274509804, 0.1411764705882353, 0.086...","[0.18823529411764706, 0.20784313725490197, 0.2...","[0.23529411764705882, 0.18823529411764706, 0.1...","[0.23921568627450981, 0.19215686274509805, 0.1...","[0.16470588235294117, 0.19607843137254902, 0.2...","[0.29411764705882354, 0.1568627450980392, 0.12...","[0.1843137254901961, 0.20392156862745098, 0.26...","[0.16470588235294117, 0.1843137254901961, 0.25...","[0.27058823529411763, 0.1568627450980392, 0.15...",2022-01-06
3,"[0.1640625, 0.1796875, 0.23046875, 0.23828125,...","[0.1953125, 0.21875, 0.203125, 0.2265625, 0.15...","[0.140625, 0.25, 0.2265625, 0.2578125, 0.125]","[0.1640625, 0.24609375, 0.1875, 0.2265625, 0.1...","[0.31640625, 0.1171875, 0.08984375, 0.12109375...","[0.1328125, 0.21875, 0.2109375, 0.2578125, 0.1...","[0.25, 0.18359375, 0.18359375, 0.15234375, 0.2...","[0.1484375, 0.18359375, 0.20703125, 0.2734375,...","[0.15625, 0.20703125, 0.25390625, 0.21484375, ...","[0.171875, 0.20703125, 0.203125, 0.2109375, 0....",...,"[0.32421875, 0.140625, 0.0859375, 0.15625, 0.2...","[0.1875, 0.20703125, 0.22265625, 0.18359375, 0...","[0.234375, 0.1875, 0.17578125, 0.203125, 0.199...","[0.2421875, 0.19140625, 0.1640625, 0.19921875,...","[0.1640625, 0.19921875, 0.23828125, 0.21875, 0...","[0.296875, 0.15625, 0.12890625, 0.109375, 0.30...","[0.18359375, 0.20703125, 0.26171875, 0.1953125...","[0.16796

In [ ]:
t2=pd.DataFrame.from_dict(t)
t2.set_index('date')

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-01-04,"[0.16600790513833993, 0.17786561264822134, 0.2...","[0.1976284584980237, 0.21739130434782608, 0.20...","[0.1422924901185771, 0.2490118577075099, 0.229...","[0.16205533596837945, 0.24110671936758893, 0.1...","[0.31225296442687744, 0.11857707509881422, 0.0...","[0.13438735177865613, 0.22134387351778656, 0.2...","[0.25296442687747034, 0.18181818181818182, 0.1...","[0.15019762845849802, 0.18181818181818182, 0.2...","[0.15810276679841898, 0.20948616600790515, 0.2...","[0.17391304347826086, 0.20948616600790515, 0.2...",...,"[0.19367588932806323, 0.1857707509881423, 0.17...","[0.3241106719367589, 0.1383399209486166, 0.086...","[0.18972332015810275, 0.20948616600790515, 0.2...","[0.233201581027668, 0.18972332015810275, 0.177...","[0.23715415019762845, 0.18972332015810275, 0.1...","[0.16205533596837945, 0.1976284584980237, 0.24...","[0.2924901185770751, 0.15810276679841898, 0.12...","[0.1857707509881423, 0.2015810276679842, 0.260...","[0.15810276679841898, 0.1857707509881423, 0.26...","[0.2727272727272727, 0.15810276679841898, 0.15..."
2022-01-05,"[0.16535433070866143, 0.17716535433070865, 0.2...","[0.1968503937007874, 0.21653543307086615, 0.20...","[0.14173228346456693, 0.24803149606299213, 0.2...","[0.16141732283464566, 0.2440944881889764, 0.18...","[0.31496062992125984, 0.11811023622047244, 0.0...","[0.13385826771653545, 0.2204724409448819, 0.21...","[0.25196850393700787, 0.18110236220472442, 0.1...","[0.14960629921259844, 0.18110236220472442, 0.2...","[0.15748031496062992, 0.20866141732283464, 0.2...","[0.1732283464566929, 0.20866141732283464, 0.20...",...,"[0.19291338582677164, 0.1889763779527559, 0.17...","[0.3228346456692913, 0.14173228346456693, 0.08...","[0.1889763779527559, 0.20866141732283464, 0.22...","[0.23622047244094488, 0.1889763779527559, 0.17...","[0.24015748031496062, 0.1889763779527559, 0.16...","[0.16141732283464566, 0.1968503937007874, 0.24...","[0.2952755905511811, 0.15748031496062992, 0.12...","[0.18503937007874016, 0.20078740157480315, 0.2...","[0.16141732283464566, 0.18503937007874016, 0.2...","[0.27165354330708663, 0.15748031496062992, 0.1..."
2022-01-06,"[0.16470588235294117, 0.1803921568627451, 0.22...","[0.19607843137254902, 0.2196078431372549, 0.20...","[0.1411764705882353, 0.25098039215686274, 0.22...","[0.16470588235294117, 0.24313725490196078, 0.1...","[0.3137254901960784, 0.11764705882352941, 0.09...","[0.13333333333333333, 0.2196078431372549, 0.21...","[0.25098039215686274, 0.1843137254901961, 0.18...","[0.14901960784313725, 0.1803921568627451, 0.20...","[0.1568627450980392, 0.20784313725490197, 0.25...","[0.17254901960784313, 0.20784313725490197, 0.2...",...,"[0.19215686274509805, 0.18823529411764706, 0.1...","[0.3215686274509804, 0.1411764705882353, 0.086...","[0.18823529411764706, 0.20784313725490197, 0.2...","[0.23529411764705882, 0.18823529411764706, 0.1...","[0.23921568627450981, 0.19215686274509805, 0.1...","[0.16470588235294117, 0.19607843137254902, 0.2...","[0.29411764705882354, 0.1568627450980392, 0.12...","[0.1843137254901961, 0.20392156862745098, 0.26...","[0.16470588235294117, 0.1843137254901961, 0.25...","[0.27058823529411763, 0.1568627450980392, 0.15..."
2022-01-07,"[0.1640625, 0.1796875, 0.23046875, 0.23828125,...","[0.1953125, 0.21875, 0.203125, 0.2265625, 0.15...","[0.140625, 0.25, 0.2265625, 0.2578125, 0.125]","[0.1640625, 0.24609375, 0.1875, 0.2265625, 0.1...","[0.31640625, 0.1171875, 0.08984375, 0.12109375...","[0.1328125, 0.21875, 0.2109375, 0.2578125, 0.1...","[0.25, 0.18359375, 0.18359375, 0.15234375, 0.2...","[0.1484375, 0.18359375, 0.20703125, 0.2734375,...","[0.15625, 0.20703125, 0.25390625, 0.21484375, ...","[0.171875, 0.20703125, 0.203125, 0.2109375, 0....",...,"[0.19140625, 0.1875, 0.17578125, 0.22265625, 0...","[0.32421875, 0.140625, 0.0859375, 0.15625, 0.2...","[0.1875, 0.20703125, 0.22265625, 0.18359375, 0...","[0.234375, 0.1875, 0.17578125, 0

In [ ]:
def create_historical(prob):
  prob=prob.rename_axis("symbol", axis="columns")
  prob=prob.T
  prob['date']=prob.columns.date[0]
  prob.columns = ['price', 'date']
  prob=prob.reset_index()
  prob=prob[['symbol','date','price']]
  return prob

In [ ]:
RPS=pd.DataFrame()

for k in t2.index: #for everyday of a stock->index
  prob=pd.DataFrame()
  prob1=pd.DataFrame()
  predic=pd.DataFrame()
  if(t2['date'][k]!=t2['date'].tail(1).values): #if it's not for the last day 
    print(t2['date'][k])
    #for historical data of two days 
    prob=rd.loc[(rd.index.date==t2['date'][k])]
    prob1=rd.loc[ (rd.index.date==t2['date'][k+1])]
    prob=create_historical(prob)
    prob1=create_historical(prob1)
    hist_data=prob.append(prob1)
    # print(hist_data)
    predic=t2.loc[t2['date']==t2['date'][k]] #find predictions of this day
    predic=predic.drop('date',axis=1)
    # print(predic)
    data=pd.DataFrame(index=predic.columns)
    data=data.reset_index()
    # print(data)
    data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
    for i in range(len(np.array_split(predic.values[0],46))-1):
      # print(i)
      # print(np.array_split(predic.values[0],46)[i][0])
      data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
    data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
    # print(data)
    #now we have data->predictions and hist_data->historical data so now we can call the RPS
    rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
    # print(rps) ->rps for every stock for one day 
    # print(rps['RPS'])
    tmp=pd.DataFrame(rps['RPS'].values).T
    # print(tmp.T)
    RPS=RPS.append(tmp,ignore_index=True)
    # print(RPS[RPS.index==k])

    
  else:
    #for the last day
    prob=rd.loc[(rd.index.date==t2['date'][k])]
    prob1=rd.loc[ (rd.index.date==rd.index.date[-1])]
    prob=create_historical(prob)
    prob1=create_historical(prob1)
    hist_data=prob.append(prob1)
    # print(hist_data)
    predic=t2.loc[t2['date']==t2['date'][k]] #find predictions of this day
    predic=predic.drop('date',axis=1)
    # print(predic)
    data=pd.DataFrame(index=predic.columns)
    data=data.reset_index()
    # print(data)
    data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
    for i in range(len(np.array_split(predic.values[0],46))-1):
      # print(i)
      # print(np.array_split(predic.values[0],46)[i][0])
      data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
    data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
    # print(data)
    #now we have data->predictions and hist_data->historical data so now we can call the RPS
    rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
    # print(rps) ->rps for every stock for one day 
    # print(rps['RPS'])
    tmp=pd.DataFrame(rps['RPS'].values).T
    # print(tmp.T)
    RPS=RPS.append(tmp,ignore_index=True)    
            
print(RPS)


2022-01-04
      ID    Return  Position Rank Rank1 Rank2 Rank3 Rank4 Rank5
21   CZR -0.064480       1.0  1.0   1.0   0.0   0.0   0.0   0.0
27  GOOG -0.046830       2.0  1.0   1.0   0.0   0.0   0.0   0.0
20  CTAS -0.045000       3.0  1.0   1.0   0.0   0.0   0.0   0.0
32  META -0.036728       4.0  1.0   1.0   0.0   0.0   0.0   0.0
41   URI -0.034775       5.0  1.0   1.0   0.0   0.0   0.0   0.0
43  VRSK -0.033464       6.0  1.0   1.0   0.0   0.0   0.0   0.0
39  ROST -0.030930       7.0  1.0   1.0   0.0   0.0   0.0   0.0
38   ROL -0.029212       8.0  1.0   1.0   0.0   0.0   0.0   0.0
4   AMAT -0.029174       9.0  1.0   1.0   0.0   0.0   0.0   0.0
23   DPZ -0.027556      10.0  2.0   0.0   1.0   0.0   0.0   0.0
14  CARR -0.026942      11.0  2.0   0.0   1.0   0.0   0.0   0.0
24   DRE -0.026204      12.0  2.0   0.0   1.0   0.0   0.0   0.0
16  CHTR -0.024830      13.0  2.0   0.0   1.0   0.0   0.0   0.0
3   ALLE -0.023338      14.0  2.0   0.0   1.0   0.0   0.0   0.0
15   CDW -0.022102      15.0 

In [ ]:
d=t2['date']
name=t2.columns[:-1]
# print(np.split(name.values,len(name)))
# print(d.values)
RPS['date']=d.values
RPS=RPS.set_index('date')
RPS.columns=assets
RPS

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-01-04,0.227449,0.263710,0.114693,0.115971,0.230446,0.100323,0.091490,0.077264,0.113840,0.113593,...,0.133512,0.136518,0.075371,0.223884,0.223678,0.109928,0.215757,0.066413,0.261916,0.228537
2022-01-05,0.135898,0.110456,0.110642,0.250493,0.195564,0.099535,0.117543,0.076657,0.067276,0.112701,...,0.108841,0.225209,0.246432,0.248896,0.117983,0.261820,0.113274,0.111560,0.259858,0.226741
2022-01-06,0.072874,0.262533,0.265855,0.114414,0.230111,0.098756,0.090740,0.144292,0.066750,0.111819,...,0.220146,0.207760,0.244503,0.126948,0.221373,0.125653,0.214348,0.110687,0.257824,0.133201
2022-01-07,0.226993,0.076111,0.254407,0.113522,0.154880,0.072986,0.116595,0.075980,0.258417,0.078134,...,0.085617,0.206140,0.122284,0.125958,0.219647,0.238736,0.131427,0.236386,0.107376,0.100912
2022-01-10,0.135736,0.232718,0.067217,0.117310,0.138113,0.226505,0.245651,0.143094,0.247071,0.077527,...,0.084952,0.226329,0.121334,0.225369,0.249070,0.070345,0.223013,0.127935,0.063741,0.224642
2022-01-11,0.072670,0.123941,0.110108,0.113301,0.195959,0.097620,0.116619,0.092374,0.113377,0.126540,...,0.084295,0.205973,0.244426,0.246881,0.219236,0.237242,0.213536,0.126945,0.257046,0.131431
2022-01-12,0.269793,0.261210,0.109259,0.112428,0.155066,0.096868,0.216878,0.091662,0.066943,0.076916,...,0.219552,0.204386,0.242543,0.119110,0.090905,0.126534,0.222701,0.066508,0.255065,0.100303
2022-01-13,0.227722,0.122281,0.066882,0.111565,0.195414,0.096124,0.138290,0.075574,0.121814,0.257864,...,0.217867,0.135124,0.240680,0.224349,0.217899,0.110178,0.213299,0.065997,0.253107,0.224148
2022-01-14,0.225980,0.121346,0.066370,0.118375,0.230701,0.283129,0.115776,0.091088,0.113219,0.076581,...,0.084400,0.134091,0.238840,0.117652,0.216233,0.070254,0.211667,0.266259,0.263431,0.222434


In [ ]:
RPS.mean(axis=1).mean()

0.158718303833372

In [ ]:
RPS.to_excel('method2_dummy.xlsx')
RPS.to_csv('method2_dummy.csv')

#####Compare with point forecast 
***

In [ ]:
t2.set_index('date',inplace=True)


In [ ]:
flag=False
for i in t2: #for every stock
  print(i)
  for j in range(len(t2[i].values)): #each day of this stock
    temp = t2[i].values[j]
    max= np.max(temp) #find max possiblity
    #replace this max possibility with 1
    temp2=np.where(temp!=max,0,temp)
    temp2=np.where(temp2==max,1,temp2)
    print(temp,'kai',temp2)
    if((temp2==1).sum()>1):
      print('isotimia')
      flag=True
      if((temp2==1).sum()==2):
        temp2=np.where(temp2==1,0.5,temp2)
        print(temp2)
      elif((temp2==1).sum()==3):
        temp2=np.where(temp2==1,0.333,temp2)
        print(temp2)
      elif((temp2==1).sum()==4):
        temp2=np.where(temp2==1,0.25,temp2)
        print(temp2)
      elif((temp2==1).sum()==5):
        temp2=np.where(temp2==1,0.2,temp2)
        print(temp2)
    t2[i].values[j]=temp2

ABBV
[0. 0. 0. 1. 0.] kai [0. 0. 0. 1. 0.]
[0. 0. 0. 1. 0.] kai [0. 0. 0. 1. 0.]
[0. 0. 0. 1. 0.] kai [0. 0. 0. 1. 0.]
[0. 0. 0. 1. 0.] kai [0. 0. 0. 1. 0.]
[0. 0. 0. 1. 0.] kai [0. 0. 0. 1. 0.]
[0. 0. 0. 1. 0.] kai [0. 0. 0. 1. 0.]
[0. 0. 0. 1. 0.] kai [0. 0. 0. 1. 0.]
[0. 0. 0. 1. 0.] kai [0. 0. 0. 1. 0.]
[0. 0. 0. 1. 0.] kai [0. 0. 0. 1. 0.]
[0. 0. 0. 1. 0.] kai [0. 0. 0. 1. 0.]
[0. 0. 0. 1. 0.] kai [0. 0. 0. 1. 0.]
[0. 0. 0. 1. 0.] kai [0. 0. 0. 1. 0.]
[0. 0. 0. 1. 0.] kai [0. 0. 0. 1. 0.]
[0. 0. 0. 1. 0.] kai [0. 0. 0. 1. 0.]
[0. 0. 0. 1. 0.] kai [0. 0. 0. 1. 0.]
[0. 0. 0. 1. 0.] kai [0. 0. 0. 1. 0.]
[0. 0. 0. 1. 0.] kai [0. 0. 0. 1. 0.]
[0. 0. 0. 1. 0.] kai [0. 0. 0. 1. 0.]
[0. 0. 0. 1. 0.] kai [0. 0. 0. 1. 0.]
[0. 0. 0. 1. 0.] kai [0. 0. 0. 1. 0.]
[0.  0.  0.5 0.5 0. ] kai [0. 0. 1. 1. 0.]
isotimia
[0.  0.  0.5 0.5 0. ]
[0.  0.  0.5 0.5 0. ] kai [0. 0. 1. 1. 0.]
isotimia
[0.  0.  0.5 0.5 0. ]
[0. 0. 0. 1. 0.] kai [0. 0. 0. 1. 0.]
[0. 0. 0. 1. 0.] kai [0. 0. 0. 1. 0.]
[0. 0. 0. 1

In [ ]:
flag

True

In [ ]:
t2=t2.reset_index()

In [ ]:
RPS=pd.DataFrame()

# RPS=RPS.drop('date',axis=1)
# print(RPS.index[2])
for k in t2.index: #for everyday of a stock->index
  prob=pd.DataFrame()
  prob1=pd.DataFrame()
  predic=pd.DataFrame()
  if(t2['date'][k]!=t2['date'].tail(1).values): #if it's not for the last day 
    #for historical data of two days 
    prob=rd.loc[(rd.index.date==t2['date'][k])]
    prob1=rd.loc[ (rd.index.date==t2['date'][k+1])]
    prob=create_historical(prob)
    prob1=create_historical(prob1)
    hist_data=prob.append(prob1)
    # print(hist_data)
    predic=t2.loc[t2['date']==t2['date'][k]] #find predictions of this day
    predic=predic.drop('date',axis=1)
    # print(predic)
    data=pd.DataFrame(index=predic.columns)
    data=data.reset_index()
    # print(data)
    data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
    for i in range(len(np.array_split(predic.values[0],46))-1):
      # print(i)
      print(np.array_split(predic.values[0],46)[i][0])
      data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
    data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
    # print(data)
    print(np.max(hist_data.date))
    #now we have data->predictions and hist_data->historical data so now we can call the RPS
    rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
    # print(rps) ->rps for every stock for one day 
    # print(rps['RPS'])
    tmp=pd.DataFrame(rps['RPS'].values).T
    # print(tmp.T)
    RPS=RPS.append(tmp,ignore_index=True)
    # print(RPS[RPS.index==k])

    
  else:
    #for the last day
    prob=rd.loc[(rd.index.date==t2['date'][k])]
    prob1=rd.loc[ (rd.index.date==rd.index.date[-1])]
    prob=create_historical(prob)
    prob1=create_historical(prob1)
    hist_data=prob.append(prob1)
    # print(hist_data)
    predic=t2.loc[t2['date']==t2['date'][k]] #find predictions of this day
    predic=predic.drop('date',axis=1)
    # print(predic)
    data=pd.DataFrame(index=predic.columns)
    data=data.reset_index()
    # print(data)
    data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
    for i in range(len(np.array_split(predic.values[0],46))-1):
      # print(i)
      # print(np.array_split(predic.values[0],46)[i][0])
      data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
    data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
    # print(data)
    
    print(hist_data)
    #now we have data->predictions and hist_data->historical data so now we can call the RPS
    rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
    
    # print(rps) ->rps for every stock for one day 
    # print(rps['RPS'])
    tmp=pd.DataFrame(rps['RPS'].values).T
    # print(tmp.T)
    RPS=RPS.append(tmp,ignore_index=True)              
print(RPS)

[0. 0. 0. 1. 0.]
[0. 0. 0. 1. 0.]
[0. 0. 0. 1. 0.]
[0. 1. 0. 0. 0.]
[0. 0. 0. 0. 1.]
[0. 0. 0. 1. 0.]
[1. 0. 0. 0. 0.]
[0. 0. 0. 1. 0.]
[0. 0. 1. 0. 0.]
[0.  0.5 0.  0.  0.5]
[0. 0. 1. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 0. 1. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 0. 1. 0. 0.]
[0.333 0.333 0.    0.    0.333]
[1. 0. 0. 0. 0.]
[0. 0. 1. 0. 0.]
[0. 0. 0. 0. 1.]
[0. 0. 0. 1. 0.]
[1. 0. 0. 0. 0.]
[0. 0. 1. 0. 0.]
[0. 0. 0. 0. 1.]
[0. 0. 1. 0. 0.]
[0. 0. 0. 0. 1.]
[0. 0. 0. 0. 1.]
[0. 0. 1. 0. 0.]
[0. 0. 0. 1. 0.]
[0. 0. 0. 1. 0.]
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 0. 0. 0. 1.]
[0. 0. 0. 1. 0.]
[0. 1. 0. 0. 0.]
[0.  0.  0.  0.5 0.5]
[1. 0. 0. 0. 0.]
[0. 0. 1. 0. 0.]
[1. 0. 0. 0. 0.]
[1. 0. 0. 0. 0.]
[0. 0. 1. 0. 0.]
[0. 0. 0. 0. 1.]
[0. 0. 1. 0. 0.]
[0. 0. 0. 1. 0.]
[1. 0. 0. 0. 0.]
2022-01-05
[0. 0. 0. 1. 0.]
[0. 0. 0. 1. 0.]
[0. 0. 0. 1. 0.]
[0. 1. 0. 0. 0.]
[0. 0. 0. 0. 1.]
[0. 0. 0. 1. 0.]
[1. 0. 0. 0. 0.]
[0. 0. 0. 1. 0.]
[0. 0. 1. 0. 0.]
[0.  0.5 0.  0.  0.5]
[0. 0. 1. 0. 0.]
[0. 1. 

In [ ]:
d=t2['date']
name=t2.columns[:-1]
# print(np.split(name.values,len(name)))
# print(d.values)
RPS['date']=d.values
RPS=RPS.set_index('date')
RPS.columns=assets
RPS

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-01-04,0.20,0.2,0.0,0.0,0.8,0.0,0.4,0.2,0.2,0.150000,...,0.45,0.2,0.0000,0.0,0.0,0.2,0.8,0.0,0.60,0.8
2022-01-05,0.40,0.4,0.4,0.2,0.0,0.0,0.2,0.2,0.0,0.150000,...,0.05,0.8,0.4000,0.8,0.2,0.4,0.4,0.2,0.60,0.8
2022-01-06,0.20,0.2,0.2,0.0,0.8,0.0,0.4,0.4,0.0,0.066667,...,0.20,0.0,0.4000,0.6,0.0,0.2,0.8,0.2,0.60,0.2
2022-01-07,0.20,0.2,0.6,0.0,0.6,0.2,0.2,0.2,0.4,0.200000,...,0.25,0.0,0.2000,0.6,0.0,0.4,0.6,0.4,0.00,0.2
2022-01-10,0.40,0.6,0.2,0.4,0.2,0.2,0.8,0.4,0.4,0.200000,...,0.25,0.8,0.2000,0.0,0.8,0.0,0.0,0.2,0.20,0.2
2022-01-11,0.20,0.0,0.4,0.0,0.0,0.0,0.2,0.0,0.2,0.250000,...,0.25,0.0,0.4000,0.8,0.0,0.4,0.8,0.2,0.60,0.6
2022-01-12,0.60,0.2,0.4,0.0,0.6,0.0,0.0,0.0,0.0,0.044489,...,0.05,0.0,0.4000,0.2,0.4,0.2,0.0,0.0,0.60,0.4
2022-01-13,0.20,0.0,0.1,0.0,0.0,0.0,0.6,0.2,0.2,0.400000,...,0.00,0.2,0.4000,0.0,0.0,0.2,0.8,0.0,0.60,0.0
2022-01-14,0.20,0.0,0.1,0.4,0.8,0.6,0.2,0.0,0.2,0.000000,...,0.40,0.2,0.4000,0.2,0.0,0.0,0.8,0.4,0.20,0.0


In [ ]:
RPS.mean(axis=1).mean()

0.2878291045185185

In [ ]:
#save results
RPS.to_excel('method2_dummy_POINT_FORECAST.xlsx')